
 # 📈 Étude du Rendement des Céréales en Algérie (2018-2019)
 ## Analyse des facteurs influençant les rendements agricoles

 ### Objectifs :
 1. Analyser les rendements des céréales par wilaya
 2. Évaluer l'impact des conditions météo
 3. Développer un modèle prédictif des rendements
 4. Identifier les zones à fort potentiel d'amélioration

## 1. Importation des bibliothèques 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Pour les visualisations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 2. Chargement et nettoyage des données agricoles

In [2]:
# Création d'un DataFrame à partir du tableau 001 (données nationales par culture)
data_agricole = {
    'Culture': ['Céréales d\'hiver', 'Blé dur', 'Blé tendre', 'Orge', 'Avoine', 'Triticale', 
                'Céréales d\'été', 'Maïs', 'Sorgho', 'Cultures industrielles', 'Tomates industrielles',
                'Tabacs', 'Arachides', 'Autres industrielles', 'Légumes secs', 'Fèves-féveroles',
                'Pois secs', 'Lentilles', 'Pois chiches', 'Haricots secs', 'Gesses et guerfalas',
                'Cultures maraîchères', 'Pommes de terre', 'Tomates maraîchères', 'Oignons', 'Ails',
                'Melons Pastèques', 'Carottes', 'Piments', 'Poivrons', 'Concombres', 'Courgettes',
                'Aubergines', 'Artichauts', 'Choux verts', 'Choux fleurs', 'Navets', 'Fèves vertes',
                'Haricots verts', 'Petits pois', 'Autres légumes'],
    'Sup_2018_ha': [3106102, 1492546, 455856, 1080250, 77375, 75, 2515, 2250, 265, 36427, 23689,
                   5097, 4753, 2888, 111823, 40222, 10383, 25956, 32065, 2843, 355, 509341,
                   149665, 22323, 47282, 12945, 60400, 17941, 10686, 11422, 4359, 13528, 5978,
                   5784, 4069, 8090, 8171, 32645, 11610, 35897, 46547],
    'Prod_2018_qx': [60568040, 31780207, 8031984, 19573271, 1180178, 2400, 450450, 414235, 36215,
                    15872668, 15406655, 106859, 118861, 240293, 1376378, 548780, 114721, 296629,
                    382736, 28113, 5400, 136570863, 46533222, 13097452, 13996907, 2022005, 20957575,
                    4318834, 3093299, 3417150, 1936474, 3888751, 1816181, 1246591, 1074125, 2076970,
                    1397446, 3086891, 931836, 1862025, 9817131],
    'Rdt_2018_qx_ha': [19.5, 21.3, 17.6, 18.1, 15.3, 32.0, 179.1, 184.1, 136.7, 435.7, 650.4,
                      21.0, 25.0, 83.2, 12.3, 13.6, 11.0, 11.4, 11.9, 9.9, 15.2, 268.1, 310.9,
                      586.7, 296.0, 156.2, 347.0, 240.7, 289.5, 299.2, 444.2, 287.5, 303.8,
                      215.5, 264.0, 256.7, 171.0, 94.6, 80.3, 51.9, 210.9],
    'Sup_2019_ha': [3185647, 1579080, 395907, 1133005, 77626, 30, 1665, 1533, 132, 38018, 24804,
                   4790, 4886, 3539, 119217, 41451, 9952, 29516, 36206, 1912, 180, 533060,
                   157864, 24994, 50292, 13403, 62652, 17469, 10348, 11381, 4078, 14372, 6027,
                   5792, 4317, 8629, 8328, 33078, 12706, 38959, 48373],
    'Prod_2019_qx': [56259902, 32087678, 6681084, 16477463, 1013052, 626, 72954, 63683, 9271,
                    17178179, 16492150, 151646, 126022, 408362, 1362293, 552729, 121250, 265242,
                    403699, 17894, 1480, 146700413, 50202499, 14778786, 16137298, 2233113, 22068666,
                    4195344, 3082823, 3668857, 1660456, 4201354, 1841457, 1196363, 1164265, 2054395,
                    1499067, 2984837, 950226, 2000258, 10780353],
    'Rdt_2019_qx_ha': [17.7, 20.3, 16.9, 14.5, 13.1, 20.9, 43.8, 41.5, 70.2, 451.8, 664.9,
                      31.7, 25.8, 115.4, 11.4, 13.3, 12.2, 9.0, 11.2, 9.4, 8.2, 275.2, 318.0,
                      591.3, 320.9, 166.6, 352.2, 240.2, 297.9, 322.4, 407.2, 292.3, 305.5,
                      206.6, 269.7, 238.1, 180.0, 90.2, 74.8, 51.3, 222.9]
}

df_cultures = pd.DataFrame(data_agricole)

# Calcul des taux d'accroissement
df_cultures['Accroissement_Rdt_%'] = ((df_cultures['Rdt_2019_qx_ha'] - df_cultures['Rdt_2018_qx_ha']) / 
                                     df_cultures['Rdt_2018_qx_ha'] * 100)

In [3]:
# Affichage des données céréalières
cereales = df_cultures[df_cultures['Culture'].str.contains('Céréales|Blé|Orge|Avoine|Triticale|Maïs|Sorgho')]
print("📊 Données des céréales (2018-2019):")
print(cereales[['Culture', 'Rdt_2018_qx_ha', 'Rdt_2019_qx_ha', 'Accroissement_Rdt_%']])

📊 Données des céréales (2018-2019):
            Culture  Rdt_2018_qx_ha  Rdt_2019_qx_ha  Accroissement_Rdt_%
0  Céréales d'hiver            19.5            17.7            -9.230769
1           Blé dur            21.3            20.3            -4.694836
2        Blé tendre            17.6            16.9            -3.977273
3              Orge            18.1            14.5           -19.889503
4            Avoine            15.3            13.1           -14.379085
5         Triticale            32.0            20.9           -34.687500
6    Céréales d'été           179.1            43.8           -75.544389
7              Maïs           184.1            41.5           -77.457903
8            Sorgho           136.7            70.2           -48.646672
